In [3]:
import math 
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

# Givens Rotation Matrix

In [6]:
# Inputs
# A : n x n symmetric matrix (A.T = A)
# p,q : row and column to zero out
# Outputs
# G : Orthogonal Matrix such that (G^T A G)_pq = (G^T A G)_qp = 0
def givens_rotation (A,p,q):
    G = np.identity(len(A))
    if (A[p,q] != 0):
        temp = (A[p,p] - A[q,q])/A[p,q]
        root = (temp + math.sqrt(temp**2+4))/2
        c = 1/math.sqrt(1+root**2)
        s = c*root
        G[p,p] = c
        G[p,q] = s
        G[q,p] = -s
        G[q,q] = c
    return (G)

# Our First Test Matrix
# $$A = \begin{bmatrix} 2 & 1 & 1 & -2 \\ 1 & 2 & 1 & 1 \\ 1 & 1 & 2 & 1 \\ -2 & 1 & 1 & 2 \end{bmatrix}$$

In [7]:
A = np.array([[2,1,1,-2],[1,2,1,1],[1,1,2,1],[-2,1,1,2]])
print (A)

[[ 2  1  1 -2]
 [ 1  2  1  1]
 [ 1  1  2  1]
 [-2  1  1  2]]


# Part 1 : Finish the function locate_max
## You Must use a nested for loop in your implementation!

In [8]:
# Inputs
# A : n x n symmetric matrix (A.T = A)
# Outputs
# max : maximum size off diagonal entry
# max_row, max_col : location of maximum size off diagonal entry
def locate_max (A):
    n = len(A)
    max_row = 0;
    max_col = 1;
    max = abs(A[max_row,max_col])
    # Add Nested For Loop here
    for i in range(0, n):
        for j in range(i+1, n): 
            temp = abs(A[i,j])
            if (temp > max):
                max = temp
                max_row = i
                max_col = j
    return [max,max_row,max_col]

## Test locate_max on $A$.  What result do you expect?

In [9]:
# I expect a max value of 2 at (python) position (0, 3)
[max, max_row, max_col] = locate_max(A)
print (max,max_row,max_col)

2 0 3


## Call givens_rotation to Find a rotation matrix to zero out the maximum off diagonal entry.

In [10]:
G = givens_rotation(A,max_row,max_col)
print (G)

[[ 0.70710678  0.          0.          0.70710678]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [-0.70710678  0.          0.          0.70710678]]


## Apply the similarity transform to zero out the maximum off diagonal entry.

In [7]:
print (G.T @ A @ G)

[[4.         0.         0.         0.        ]
 [0.         2.         1.         1.41421356]
 [0.         1.         2.         1.41421356]
 [0.         1.41421356 1.41421356 0.        ]]


## Test locate_max on $G^T A G$.  What result do you expect?  

In [8]:
# I expect the max to be less than before so less than 2
[max, max_row, max_col] = locate_max(G.T @ A @ G)
print (max,max_row,max_col)

1.414213562373095 1 3


# Part 2 : Finish the function jacobi_factor

In [37]:
# Inputs
# A : n x n symmetric matrix (A.T = A)
# max_steps : Maximum Number of Iterations
# eps : Converges when all off diagonal entries of D have size <= eps
# Outputs
# flag : TRUE if successful and FALSE otherwise
# num : Number of Iterations
# D, U : If flag==TRUE, then D is diagonal, U^T U = I, and A = UDU^T
def jacobi_factor (A,max_steps,eps):
    n = len(A)
    D = A
    U = np.identity(n)
    num = 0
    [max,p,q] = locate_max(D)
    # Add While Loop Here
    while(max > eps and num < max_steps):
        G = givens_rotation(D, p, q)
        U = U@G
        D = U.T @ A @ U
        num = num + 1
        [max, p , q] = locate_max(D)
    return [max <= eps,num,D,U]

## Test the jacobi_factor function.  

* Does your algorithm converge?
* How many iterations were required?
* Do your computed eigenvalues agree with the built-in Python function?
* Is your $U$ matrix orthogonal?
* Does $A = UDU^T$?

In [21]:
print (linalg.eig(A)[0].real)

[ 4. -1.  4.  1.]


In [39]:
[flag,num,D,U] = jacobi_factor(A,100,1e-6)
print (flag,num,'\n\n',D,'\n\n',U,'\n\n',U.T @ U,'\n\n',U @ D @ U.T)

True 9 

 [[ 4.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  4. -0.]
 [ 0.  0. -0. -1.]] 

 [[ 0.70710678  0.         -0.31622777 -0.63245553]
 [ 0.          0.70710678 -0.63245553  0.31622777]
 [ 0.         -0.70710678 -0.63245553  0.31622777]
 [-0.70710678  0.         -0.31622777 -0.63245553]] 

 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 

 [[ 2.  1.  1. -2.]
 [ 1.  2.  1.  1.]
 [ 1.  1.  2.  1.]
 [-2.  1.  1.  2.]]


In [ ]:
# Yes it converges after 9 iterations
# Yes they are the same
# Yes it is U.t @ U = I
# Yes A = UDU.T 

# Part 3 : A Second Test Matrix

In [40]:
# Create a 8x8 symmetric test matrix.
M = np.array([[-4,-3,2,1,3,4,2,1],
            [1,-2,3,4,5,1,2,3],
            [2,1,4,-3,5,2,1,-1],
            [1,4,3,-2,-1,2,-2,1],
            [2,3,-1,4,1,-3,4,2],
            [1,4,-2,3,-1,5,-2,2],
            [-1,-2,4,2,-4,3,-3,1],
            [4,2,-1,-3,3,-4,1,2]])
A = M.T @ M 
print (A)

[[ 44  36  -6 -11  19 -29   7  11]
 [ 36  63 -17  -8  -5  -7  -5  10]
 [ -6 -17  60  -3  20  34  -5   6]
 [-11  -8  -3  68  -6  19  12  24]
 [ 19  -5  20  -6  87  -7  44  14]
 [-29  -7  34  19  -7  84 -27   6]
 [  7  -5  -5  12  44 -27  43   8]
 [ 11  10   6  24  14   6   8  25]]


## Test the jacobi_factor function on the second matrix.  
* Does your algorithm converge?
* How many iterations were required?
* Do your computed eigenvalues agree with the built-in Python function?

In [41]:
print (np.sort(linalg.eig(A)[0].real))

[  0.53405422   6.23152918  11.420927    29.862678    79.03473434
  81.2045581  120.92490621 144.78661295]


In [42]:
[flag,num,D,U] = jacobi_factor(A,10000,1e-6)
print (flag,num,'\n\n',np.sort(np.diag(D)))

True 82 

 [  0.53405422   6.23152918  11.420927    29.862678    79.03473434
  81.2045581  120.92490621 144.78661295]


In [ ]:
# Yes it converges 
# It took 82 iterations
# Yes all of the eigenvalues match up